In [25]:
import pandas as pd
from glob import glob
from pykalman import KalmanFilter
import plotly.express as px
import os
import statsmodels.api as sm 
pd.set_option("display.max_rows", None)
from plotly.io import write_html

In [26]:
save_fig = False # 要不要存圖片

In [27]:
def kalman_filter(
    df,
    observation_covariance = 1,
    initial_value_guess = 0,
    transition_matrix = 1,
    transition_covariance = 0.1
):
    kf = KalmanFilter(
        initial_state_mean=initial_value_guess,
        initial_state_covariance=observation_covariance,
        observation_covariance=observation_covariance,
        transition_covariance=transition_covariance,
        transition_matrices=transition_matrix
    )
    pred_state, state_cov = kf.filter(df['linear_acceleration.y'].to_numpy())
    return pred_state

### 先做卡爾曼濾波，再用絕對值

In [29]:
result_df = pd.DataFrame(columns=['action', 'sheet', 'kalman_abs'])
for filename in glob('clean_data/*.xlsx'):
    print('Filename:{}'.format(filename))
    for i in range(10):
        try:
            df = pd.read_excel(filename, sheet_name=i, header=0, usecols=[5])
        except:
            continue
        df = df[~df['linear_acceleration.y'].isnull()]
        df['kalman_filter'] = kalman_filter(df)
        if save_fig:
            plot_df = pd.melt(df, value_vars=df.columns, ignore_index=False)
            fig = px.line(plot_df, x=plot_df.index , y='value', color='variable', title='卡爾曼濾波器')
            fig.write_html('output/{}_工作表{}.html'.format(os.path.basename(filename)[:-5], (i+1)))
        result = df['kalman_filter'].abs().sum()
        action = filename.split('走')[1].split('米')[0]
        row_df = pd.DataFrame([[action, i+1, result]], columns=['action', 'sheet', 'kalman_abs'])
        result_df = pd.concat([result_df, row_df])
result_df = result_df.reset_index(drop=True)

Filename:clean_data\直線走10米-共十次.xlsx
Filename:clean_data\直線走15米-共十次.xlsx
Filename:clean_data\直線走1米-共十次.xlsx
Filename:clean_data\直線走20米-共十次.xlsx
Filename:clean_data\直線走25米-共十次.xlsx
Filename:clean_data\直線走2米-共十次.xlsx
Filename:clean_data\直線走30米-共十次.xlsx
Filename:clean_data\直線走35米-共十次.xlsx
Filename:clean_data\直線走3米-共十次.xlsx
Filename:clean_data\直線走40米-共十次.xlsx
Filename:clean_data\直線走45米-共十次.xlsx
Filename:clean_data\直線走4米-共十次.xlsx
Filename:clean_data\直線走50米-共十次-最終資料彙集處.xlsx
Filename:clean_data\直線走5米-共十次.xlsx
Filename:clean_data\直線走6米-共十次.xlsx
Filename:clean_data\直線走7米-共十次.xlsx
Filename:clean_data\直線走8米-共十次.xlsx
Filename:clean_data\直線走9米-共十次.xlsx


In [30]:
result_df['action'] = pd.to_numeric(result_df['action'])
result_df = result_df.sort_values(by='action')
fig = px.scatter(x=result_df['action'], y=result_df['kalman_abs'])
write_html(fig, 'plot.html')
with open('table.html', 'w') as f:
    f.write(result_df.to_html())

### 平均十次結果

In [13]:
summary_df = result_df.groupby(by='action').mean()
summary_df['std'] = result_df.groupby(by='action')['kalman_abs'].std()
summary_df['max'] = result_df.groupby(by='action')['kalman_abs'].max()
summary_df['min'] = result_df.groupby(by='action')['kalman_abs'].min()
summary_df = summary_df.sort_values(by='kalman_abs')
summary_df

,kalman_abs,std,max,min
action,,,,
直線走1米,10.093323,1.552107,13.254710,7.687889
直線走3米,18.876752,1.809566,22.217636,17.030305
直線走4米,19.567598,2.464685,24.142040,17.432625
直線走2米,20.471586,10.279505,49.104462,13.714089
直線走6米,21.028888,2.964419,24.456257,16.227407
直線走5米,23.055169,4.642790,31.825743,16.063622
直線走8米,24.484172,4.039445,30.261336,17.440790
直線走7米,24.585102,3.373182,31.948465,21.359590
直線走9米,26.122839,2.355800,29.766884,23.520921


### 加入實際值

In [15]:
# summary_df['actual'] = [6, 7, 8, 9, 15, 20, 25, 30, 35, 40, 45, 50]
summary_df['actual'] = [1, 3, 4, 2, 6, 5, 8, 7, 9 ,10, 15, 20, 25, 30, 35, 40, 45, 50]
summary_df

,kalman_abs,std,max,min,actual
action,,,,,
直線走1米,10.093323,1.552107,13.254710,7.687889,1
直線走3米,18.876752,1.809566,22.217636,17.030305,3
直線走4米,19.567598,2.464685,24.142040,17.432625,4
直線走2米,20.471586,10.279505,49.104462,13.714089,2
直線走6米,21.028888,2.964419,24.456257,16.227407,6
直線走5米,23.055169,4.642790,31.825743,16.063622,5
直線走8米,24.484172,4.039445,30.261336,17.440790,8
直線走7米,24.585102,3.373182,31.948465,21.359590,7
直線走9米,26.122839,2.355800,29.766884,23.520921,9


### 迴歸分析

In [16]:
X = summary_df['kalman_abs'].ravel()
y = summary_df['actual'].ravel()
model = sm.OLS(y, X)
result = model.fit()
print(result.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.968
Model:                            OLS   Adj. R-squared (uncentered):              0.966
Method:                 Least Squares   F-statistic:                              515.7
Date:                Thu, 06 Jan 2022   Prob (F-statistic):                    3.72e-14
Time:                        06:48:26   Log-Likelihood:                         -51.314
No. Observations:                  18   AIC:                                      104.6
Df Residuals:                      17   BIC:                                      105.5
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

D:\anaconda3\envs\imu\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


### 預測

In [17]:
summary_df['predict'] = result.predict(X)
summary_df['predict_max'] = result.predict(summary_df['max'])
summary_df['predict_min'] = result.predict(summary_df['min'])
summary_df

,kalman_abs,std,max,min,actual,predict,predict_max,predict_min
action,,,,,,,,
直線走1米,10.093323,1.552107,13.254710,7.687889,1,4.864855,6.388604,3.705466
直線走3米,18.876752,1.809566,22.217636,17.030305,3,9.098358,10.708622,8.208393
直線走4米,19.567598,2.464685,24.142040,17.432625,4,9.431337,11.636161,8.402307
直線走2米,20.471586,10.279505,49.104462,13.714089,2,9.867048,23.667735,6.610019
直線走6米,21.028888,2.964419,24.456257,16.227407,6,10.135660,11.787609,7.821407
直線走5米,23.055169,4.642790,31.825743,16.063622,5,11.112302,15.339609,7.742464
直線走8米,24.484172,4.039445,30.261336,17.440790,8,11.801064,14.585585,8.406242
直線走7米,24.585102,3.373182,31.948465,21.359590,7,11.849711,15.398760,10.295055
直線走9米,26.122839,2.355800,29.766884,23.520921,9,12.590881,14.347265,11.336789


### 計算誤差

In [18]:
summary_df['mean_error'] = (summary_df['actual'] - summary_df['predict']).abs()
summary_df['max_error'] = (summary_df['actual'] - summary_df['predict_max']).abs().combine((summary_df['actual'] - summary_df['predict_min']).abs(), max)
summary_df

,kalman_abs,std,max,min,actual,predict,predict_max,predict_min,mean_error,max_error
action,,,,,,,,,,
直線走1米,10.093323,1.552107,13.254710,7.687889,1,4.864855,6.388604,3.705466,3.864855,5.388604
直線走3米,18.876752,1.809566,22.217636,17.030305,3,9.098358,10.708622,8.208393,6.098358,7.708622
直線走4米,19.567598,2.464685,24.142040,17.432625,4,9.431337,11.636161,8.402307,5.431337,7.636161
直線走2米,20.471586,10.279505,49.104462,13.714089,2,9.867048,23.667735,6.610019,7.867048,21.667735
直線走6米,21.028888,2.964419,24.456257,16.227407,6,10.135660,11.787609,7.821407,4.135660,5.787609
直線走5米,23.055169,4.642790,31.825743,16.063622,5,11.112302,15.339609,7.742464,6.112302,10.339609
直線走8米,24.484172,4.039445,30.261336,17.440790,8,11.801064,14.585585,8.406242,3.801064,6.585585
直線走7米,24.585102,3.373182,31.948465,21.359590,7,11.849711,15.398760,10.295055,4.849711,8.398760
直線走9米,26.122839,2.355800,29.766884,23.520921,9,12.590881,14.347265,11.336789,3.590881,5.347265
